Predicting total money spent on treating different cancers based on initial costs and patient demographics

In [ ]:
# https://www.kaggle.com/rishidamarla/costs-for-cancer-treatment
# https://data.world/xprizeai-health/expenditures-for-cancer-care

In [2]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


In [8]:
df

,Cost of Cancer Care by Phase of Care,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Costs ($) per year in millions of 2010 dollars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,See http://costprojections.cancer.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and l...,Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate",0%,124565.6,40463.5,46642.8,37459.2
4,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,122420.8,38552.7,46671.9,37196.3
...,...,...,...,...,...,...,...,...,...,...
1252,Other,2020,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,26945.7,4432.4,14156.5,8356.8
1253,Other,2020,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,29975.9,4466.2,17675.7,7834.0
1254,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,30892.3,4432.4,17275.7,9184.3
1255,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,33201.2,5403.1,17275.7,10522.5


In [12]:
df = pd.read_csv('DowloadableDataFull_2011.01.12.csv')
df.head(5)

,Cost of Cancer Care by Phase of Care,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Costs ($) per year in millions of 2010 dollars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,See http://costprojections.cancer.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and l...,Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate",0%,124565.6,40463.5,46642.8,37459.2
4,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,122420.8,38552.7,46671.9,37196.3


#### Cleaning

In [13]:
#
df = df.iloc[2:]

#
df.columns = df.iloc[0]

#
df = df.drop(df.index[0])

#
df = df.reset_index(drop=True)

,Cost of Cancer Care by Phase of Care,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and l...,Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate",0%,124565.6,40463.5,46642.8,37459.2
4,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,122420.8,38552.7,46671.9,37196.3
5,AllSites,2010,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,125397.7,40463.5,47136.3,37797.9
6,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,123236.3,38552.7,47155.7,37527.8
...,...,...,...,...,...,...,...,...,...,...
1252,Other,2020,Both sexes,All ages,"Incidence follows recent trend, Survival constant",0%,26945.7,4432.4,14156.5,8356.8
1253,Other,2020,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,29975.9,4466.2,17675.7,7834.0
1254,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,30892.3,4432.4,17275.7,9184.3
1255,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends",2%,33201.2,5403.1,17275.7,10522.5


In [23]:
df.sample(10)

2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
1188,Kidney,2020,Both sexes,All ages,"Incidence, Survival at constant rate",0%,5115.7,1392.2,2212.3,1511.2
335,Uterus,2012,Both sexes,All ages,"Incidence, Survival follow recent trends",5%,2760.4,1137.8,791.8,830.8
338,Other,2012,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,21667.5,4115.5,12110.1,5441.9
741,Leukemia,2016,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,6255.7,820.5,2475.6,2959.6
1226,Pancreas,2020,Both sexes,All ages,"Survival follows recent trend, Incidence constant",0%,3164.6,1736.9,228.5,1199.2
1037,Bladder,2019,Both sexes,All ages,"Incidence, Survival follow recent trends",5%,5474.9,1559.2,2144.2,1771.5
342,AllSites,2013,Both sexes,All ages,"Incidence, Survival at constant rate",0%,134173.6,42703.9,50949.8,40519.8
519,Lung,2014,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,11873.9,4861.9,1944.8,5067.3
969,Leukemia,2018,Both sexes,All ages,"Incidence, Survival follow recent trends",0%,6591.3,841.6,2619.2,3130.5
407,Lung,2013,Both sexes,All ages,"Incidence, Survival follow recent trends",5%,13261.8,5596.3,1917.0,5748.5


In [46]:
df.isna().sum()

2
Cancer Site                                                  0
Year                                                         0
Sex                                                          0
Age                                                          0
Incidence and Survival Assumptions                           0
Annual Cost Increase (applied to initial and last phases)    0
Total Costs                                                  0
Initial Year After Diagnosis Cost                            0
Continuing Phase Cost                                        0
Last Year of Life Cost                                       0
dtype: int64

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254 entries, 0 to 1253
Data columns (total 10 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   Cancer Site                                                1254 non-null   object
 1   Year                                                       1254 non-null   object
 2   Sex                                                        1254 non-null   object
 3   Age                                                        1254 non-null   object
 4   Incidence and Survival Assumptions                         1254 non-null   object
 5   Annual Cost Increase (applied to initial and last phases)  1254 non-null   object
 6   Total Costs                                                1254 non-null   object
 7   Initial Year After Diagnosis Cost                          1254 non-null   object
 8   Continuing Phase C

In [39]:
df.iloc[:, [0,1,2,3,4]]

2,Cancer Site,Year,Sex,Age,Incidence and Survival Assumptions
0,AllSites,2010,Both sexes,All ages,"Incidence, Survival at constant rate"
1,AllSites,2010,Both sexes,All ages,"Incidence follows recent trend, Survival constant"
2,AllSites,2010,Both sexes,All ages,"Survival follows recent trend, Incidence constant"
3,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends"
4,AllSites,2010,Both sexes,All ages,"Incidence, Survival follow recent trends"
...,...,...,...,...,...
1249,Other,2020,Both sexes,All ages,"Incidence follows recent trend, Survival constant"
1250,Other,2020,Both sexes,All ages,"Survival follows recent trend, Incidence constant"
1251,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends"
1252,Other,2020,Both sexes,All ages,"Incidence, Survival follow recent trends"


In [42]:
cats = list(df.columns[0:5])
cats

['Cancer Site', 'Year', 'Sex', 'Age', 'Incidence and Survival Assumptions']

In [60]:
nums = list(df.columns[5:])
nums

['Annual Cost Increase (applied to initial and last phases)',
 'Total Costs',
 'Initial Year After Diagnosis Cost',
 'Continuing Phase Cost',
 'Last Year of Life Cost']

In [45]:
for i in cats:
    print(df[i].value_counts())

Head_Neck     66
Brain         66
Ovary         66
Prostate      66
Breast        66
Leukemia      66
Kidney        66
Esophagus     66
Lung          66
Cervix        66
Lymphoma      66
Uterus        66
Stomach       66
Melanoma      66
AllSites      66
Colorectal    66
Bladder       66
Other         66
Pancreas      66
Name: Cancer Site, dtype: int64
2016    114
2015    114
2012    114
2020    114
2011    114
2019    114
2010    114
2014    114
2017    114
2013    114
2018    114
Name: Year, dtype: int64
Both sexes    990
Females       198
Males          66
Name: Sex, dtype: int64
All ages    1254
Name: Age, dtype: int64
Incidence, Survival follow recent trends             627
Incidence, Survival at constant rate                 209
Incidence follows recent trend, Survival constant    209
Survival follows recent trend, Incidence constant    209
Name: Incidence and Survival Assumptions, dtype: int64


In [49]:
df.shape

(1254, 10)

In [64]:
print(df['Annual Cost Increase (applied to initial and last phases)'].value_counts())
print(df.loc[10, 'Annual Cost Increase (applied to initial and last phases)'][:-1])
for i in range(len(df)):
    df.loc[i, 'Annual Cost Increase (applied to initial and last phases)'] = df.loc[i, 'Annual Cost Increase (applied to initial and last phases)'][:-1]

0%    836
5%    209
2%    209
Name: Annual Cost Increase (applied to initial and last phases), dtype: int64

In [72]:
print(df.loc[10, 'Annual Cost Increase (applied to initial and last phases)'])
print(df['Annual Cost Increase (applied to initial and last phases)'].value_counts())

'2'

In [77]:
for i in nums:
    df[i] = df[i].astype(float)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254 entries, 0 to 1253
Data columns (total 10 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Cancer Site                                                1254 non-null   object 
 1   Year                                                       1254 non-null   object 
 2   Sex                                                        1254 non-null   object 
 3   Age                                                        1254 non-null   object 
 4   Incidence and Survival Assumptions                         1254 non-null   object 
 5   Annual Cost Increase (applied to initial and last phases)  1254 non-null   float64
 6   Total Costs                                                1254 non-null   float64
 7   Initial Year After Diagnosis Cost                          1254 non-null   float64
 8   Continui

In [79]:
df.iloc[:, 5:].describe()

2,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
count,1254.000000,1254.000000,1254.000000,1254.000000,1254.000000
mean,1.166667,15221.008852,4677.505742,5764.655024,4778.848804
std,1.864133,31489.503603,9703.258460,12094.699099,9930.281985
min,0.000000,1204.600000,293.000000,98.900000,330.600000
25%,0.000000,2866.625000,939.175000,846.875000,908.925000
50%,0.000000,5107.350000,1377.150000,1940.350000,1755.200000
75%,2.000000,14178.925000,4883.050000,5187.200000,4465.275000
max,5.000000,206587.100000,68858.000000,66099.700000,74159.700000


In [ ]:
df.iloc[:, 5:].corr()

In [82]:
df['Age'].value_counts()

All ages    1254
Name: Age, dtype: int64

In [83]:
df.drop('Age', axis=1, inplace=True)

In [84]:
df.head(4)

2,Cancer Site,Year,Sex,Incidence and Survival Assumptions,Annual Cost Increase (applied to initial and last phases),Total Costs,Initial Year After Diagnosis Cost,Continuing Phase Cost,Last Year of Life Cost
0,AllSites,2010,Both sexes,"Incidence, Survival at constant rate",0.0,124565.6,40463.5,46642.8,37459.2
1,AllSites,2010,Both sexes,"Incidence follows recent trend, Survival constant",0.0,122420.8,38552.7,46671.9,37196.3
2,AllSites,2010,Both sexes,"Survival follows recent trend, Incidence constant",0.0,125397.7,40463.5,47136.3,37797.9
3,AllSites,2010,Both sexes,"Incidence, Survival follow recent trends",0.0,123236.3,38552.7,47155.7,37527.8


#### Exploring the Data

Plots

Feature Transformation and Engineering